In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get('API_KEY')

In [2]:
from google import genai

client = genai.Client(api_key=API_KEY)

## 2-agent Fake News Detection Framework

- **Agent 1**: Take the input news piece and get a list of relevant facts
- **Agent 2**: Reason with the input and the list of facts to decide whether news piece is fake

![workflow](workflow.png "Workflow of our simple 2-agent fake news detection framework")

### Agent 1: FactBase

1. We will use Gemini 1.5 Pro's better language understanding and enhanced knowledge base to get a decent base of facts.
2. The facts will be returned as CSV, which will then be passed to Agent 2

In [3]:
import pandas as pd
import io

def agent1_factbase(client, news_text):
    """
    Agent 1: Extract relevant facts from the input news using Gemini 1.5 Pro
    Returns facts in CSV format to be processed by Agent 2
    """
    prompt = f"""
    You are a fact-checking assistant. Given a news piece, extract a list of factual claims that can be verified.
    For each claim, provide:
    1. The claim
    2. The source of the claim (if mentioned in the text)
    3. A confidence score (0-1) on how verifiable this claim is based on specificity
    
    Format your output as CSV with headers: claim,source,confidence
    Do not include any other text, explanations, or formatting - ONLY the CSV data.
    
    News piece to analyze:
    {news_text}
    """
    
    response = client.models.generate_content(
        model='gemini-1.5-pro',
        contents=prompt
    )
    
    # Return the CSV string
    return response.text

### Agent 2: Verifier

1. We will use the new Gemini 2.0 Flash model with zero-shot reasoning to get the final response: fake news or not.
2. The final answer needs be returned in an XML format. This helps in easy integration with an existing code base.

In [4]:
def agent2_verifier(client, news_text, facts_csv):
    """
    Agent 2: Analyze the news and extracted facts to determine if the news is fake
    Uses Gemini 2.0 Flash for faster reasoning
    Returns analysis in XML format
    """
    # Parse the CSV into a more readable format for the prompt
    try:
        facts_df = pd.read_csv(io.StringIO(facts_csv))
        facts_formatted = "\n".join([
            f"- Claim: {row['claim']}\n  Source: {row['source']}\n  Confidence: {row['confidence']}"
            for _, row in facts_df.iterrows()
        ])
    except:
        # Fallback if CSV parsing fails
        facts_formatted = facts_csv
    
    prompt = f"""
    You are a news verification assistant. Analyze the provided news piece and fact-check results to determine if the news is likely fake or genuine.
    
    News piece:
    {news_text}
    
    Extracted facts and claims:
    {facts_formatted}
    
    Analyze the news piece with the following considerations:
    1. Do the extracted facts align with established knowledge?
    2. Are there logical inconsistencies in the text?
    3. Is the source reliable based on the extracted information?
    4. Does the writing use manipulative or emotional language?
    5. Are there any verifiable false claims?
    
    Provide your analysis in the following XML format, with no additional text before or after:
    <verification>
      <overall_assessment>1 for REAL or 0 for FAKE</overall_assessment>
      <confidence_score>0-1 score</confidence_score>
      <key_issues>List up to 3 key issues or red flags if any</key_issues>
      <reasoning>Brief explanation of your reasoning</reasoning>
    </verification>
    """
    
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt
    )
    
    return response.text

### Orchestrator

In [5]:
def detect_fake_news(client, news_text):
    """
    Main function to orchestrate the 2-agent system for fake news detection
    """
    # Step 1: Use Agent 1 to extract facts
    facts_csv = agent1_factbase(client, news_text)
    
    # Step 2: Use Agent 2 to verify and reason about the news
    verification_result = agent2_verifier(client, news_text, facts_csv)
    
    return {
        "facts_extracted": facts_csv,
        "verification": verification_result
    }

In [6]:
def run_detection_pipeline(client, news_text):
    """
    Run the complete fake news detection pipeline and display the results
    """
    print("🔍 FAKE NEWS DETECTION ANALYSIS")
    print("=" * 50)
    print(f"Analyzing news:\n{news_text[:300]}{'...' if len(news_text) > 300 else ''}\n")
    
    # Run the detection
    results = detect_fake_news(client, news_text)
    
    # Display the results
    print("📊 EXTRACTED FACTS (AGENT 1)")
    print("-" * 50)
    print(results["facts_extracted"])
    print("\n🔎 VERIFICATION RESULTS (AGENT 2)")
    print("-" * 50)
    print(results["verification"])
    
    return results

## Taking our FND framework for a spin

In [7]:
sample_news = """
    Scientists at MIT have discovered a new type of battery that can be fully charged in 30 seconds and 
    lasts for a week of heavy use. The breakthrough, published yesterday in Nature, uses a revolutionary 
    graphene-based material that could make current lithium-ion batteries obsolete within two years.
    """
    
run_detection_pipeline(client, sample_news)

🔍 FAKE NEWS DETECTION ANALYSIS
Analyzing news:

    Scientists at MIT have discovered a new type of battery that can be fully charged in 30 seconds and 
    lasts for a week of heavy use. The breakthrough, published yesterday in Nature, uses a revolutionary 
    graphene-based material that could make current lithium-ion batteries obsolete withi...

📊 EXTRACTED FACTS (AGENT 1)
--------------------------------------------------
claim,source,confidence
Scientists at MIT have discovered a new type of battery,,"0.9"
The battery can be fully charged in 30 seconds,,"0.9"
The battery lasts for a week of heavy use,,"0.7"
The breakthrough was published yesterday in Nature,Nature,0.9
The battery uses a graphene-based material,,"0.9"
The new battery could make current lithium-ion batteries obsolete within two years,,"0.5"


🔎 VERIFICATION RESULTS (AGENT 2)
--------------------------------------------------
```xml
<verification>
  <overall_assessment>0</overall_assessment>
  <confidence_score>0.8<

{'facts_extracted': 'claim,source,confidence\nScientists at MIT have discovered a new type of battery,,"0.9"\nThe battery can be fully charged in 30 seconds,,"0.9"\nThe battery lasts for a week of heavy use,,"0.7"\nThe breakthrough was published yesterday in Nature,Nature,0.9\nThe battery uses a graphene-based material,,"0.9"\nThe new battery could make current lithium-ion batteries obsolete within two years,,"0.5"\n',
 'verification': "```xml\n<verification>\n  <overall_assessment>0</overall_assessment>\n  <confidence_score>0.8</confidence_score>\n  <key_issues>\n    <issue>Extraordinary claim about battery charging speed and longevity.</issue>\n    <issue>Overly optimistic prediction of lithium-ion battery obsolescence.</issue>\n    <issue>Lack of specific researcher or research group mentioned at MIT.</issue>\n  </key_issues>\n  <reasoning>While MIT is a reputable institution and Nature is a reputable journal, the claims about the battery's performance (30-second charging and a week

## Getting results for a small sample (500) from Uni-Fakeddit-55k

In [10]:
import pandas as pd
import io
import time
from tqdm import tqdm
import csv
import os

def agent1_factbase(client, news_text):
    """
    Agent 1: Extract relevant facts from the input news using Gemini 1.5 Pro
    Returns facts in CSV format to be processed by Agent 2
    """
    prompt = f"""
    You are a fact-checking assistant. Given a news piece, extract a list of factual claims that can be verified.
    For each claim, provide:
    1. The claim
    2. The source of the claim (if mentioned in the text)
    3. A confidence score (0-1) on how verifiable this claim is based on specificity
    
    Format your output as CSV with headers: claim,source,confidence
    Do not include any other text, explanations, or formatting - ONLY the CSV data.
    
    News piece to analyze:
    {news_text}
    """
    
    try:
        response = client.models.generate_content(
            model='gemini-1.5-pro',
            contents=prompt
        )
        return response.text
    except Exception as e:
        print(f"Error in Agent 1: {e}")
        # Return a simple CSV with error information
        return "claim,source,confidence\nError processing input,agent1,0.0"

def agent2_verifier(client, news_text, facts_csv):
    """
    Agent 2: Analyze the news and extracted facts to determine if the news is fake
    Uses Gemini 2.0 Flash for faster reasoning
    Returns analysis in XML format with binary classification (1=REAL, 0=FAKE)
    """
    # Parse the CSV into a more readable format for the prompt
    try:
        facts_df = pd.read_csv(io.StringIO(facts_csv))
        facts_formatted = "\n".join([
            f"- Claim: {row['claim']}\n  Source: {row['source']}\n  Confidence: {row['confidence']}"
            for _, row in facts_df.iterrows()
        ])
    except:
        # Fallback if CSV parsing fails
        facts_formatted = facts_csv
    
    prompt = f"""
    You are a news verification assistant. Analyze the provided news piece and fact-check results to determine if the news is likely fake or genuine.
    
    News piece:
    {news_text}
    
    Extracted facts and claims:
    {facts_formatted}
    
    Analyze the news piece with the following considerations:
    1. Do the extracted facts align with established knowledge?
    2. Are there logical inconsistencies in the text?
    3. Is the source reliable based on the extracted information?
    4. Does the writing use manipulative or emotional language?
    5. Are there any verifiable false claims?
    
    Provide your analysis in the following XML format, with no additional text before or after:
    <verification>
      <overall_assessment>1 for REAL or 0 for FAKE</overall_assessment>
      <confidence_score>0-1 score</confidence_score>
      <key_issues>List up to 3 key issues or red flags if any</key_issues>
      <reasoning>Brief explanation of your reasoning</reasoning>
    </verification>
    """
    
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=prompt
        )
        return response.text
    except Exception as e:
        print(f"Error in Agent 2: {e}")
        # Return a simple XML with error information
        return "<verification>\n  <overall_assessment>0</overall_assessment>\n  <confidence_score>0.0</confidence_score>\n  <key_issues>Error processing input</key_issues>\n  <reasoning>Agent 2 encountered an error</reasoning>\n</verification>"

def detect_fake_news(client, news_text):
    """
    Main function to orchestrate the 2-agent system for fake news detection
    """
    # Step 1: Use Agent 1 to extract facts
    facts_csv = agent1_factbase(client, news_text)
    
    # Step 2: Use Agent 2 to verify and reason about the news
    verification_result = agent2_verifier(client, news_text, facts_csv)
    
    return {
        "facts_extracted": facts_csv,
        "verification": verification_result
    }

def parse_dataset_line(line):
    """
    Parse a line from the Uni-Fakkedit-55k dataset
    Returns a tuple of (text, objects, label) or None if parsing fails
    """
    try:
        # Extract text
        text_parts = line.split("[TEXT]")
        if len(text_parts) != 2:
            return None
        after_text = text_parts[1]

        # Extract objects
        text_objects_parts = after_text.split("[OBJECTS]")
        if len(text_objects_parts) != 2:
            return None
        text_part = text_objects_parts[0].strip()
        after_objects = text_objects_parts[1]

        # Extract label
        objects_label_parts = after_objects.split("[LABEL]")
        if len(objects_label_parts) != 2:
            return None
        objects_part = objects_label_parts[0].strip()
        label_part = objects_label_parts[1].strip()

        # Combine text and objects
        combined_input = f"{text_part}. The image contains: {objects_part}"
        label_int = int(label_part)
        
        return (combined_input, label_int)
    except Exception as e:
        print(f"Error parsing line: {e}")
        return None

def extract_assessment(verification_xml):
    """
    Extract the overall assessment (1 or 0) from the verification XML
    Returns the assessment value or None if extraction fails
    """
    try:
        if "<overall_assessment>1</overall_assessment>" in verification_xml:
            return 1
        elif "<overall_assessment>0</overall_assessment>" in verification_xml:
            return 0
        else:
            # Try to extract using a more flexible approach
            start_tag = "<overall_assessment>"
            end_tag = "</overall_assessment>"
            start_pos = verification_xml.find(start_tag)
            if start_pos == -1:
                return None
            start_pos += len(start_tag)
            end_pos = verification_xml.find(end_tag, start_pos)
            if end_pos == -1:
                return None
            assessment_str = verification_xml[start_pos:end_pos].strip()
            if assessment_str == "1" or assessment_str.upper() == "REAL":
                return 1
            elif assessment_str == "0" or assessment_str.upper() == "FAKE":
                return 0
            else:
                return None
    except Exception as e:
        print(f"Error extracting assessment: {e}")
        return None

def process_dataset(client, dataset_path, output_path, num_samples=500, batch_size=10):
    """
    Process the first num_samples entries from the Uni-Fakkedit-55k dataset
    Saves results to a CSV file and returns evaluation metrics
    """
    results = []
    correct_predictions = 0
    total_processed = 0
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Open output file for writing results as we go
    with open(output_path, 'w', newline='') as csvfile:
        fieldnames = ['text', 'true_label', 'predicted_label', 'correct', 'facts', 'verification']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # Process the dataset
        with open(dataset_path, 'r', encoding='utf-8') as f:
            lines = []
            for _ in range(num_samples):
                line = f.readline()
                if not line:
                    break  # End of file
                lines.append(line)
            
            # Process in batches to avoid rate limiting
            for i in tqdm(range(0, len(lines), batch_size), desc="Processing dataset"):
                batch_lines = lines[i:i+batch_size]
                batch_results = []
                
                for line in batch_lines:
                    parsed_data = parse_dataset_line(line)
                    if parsed_data is None:
                        continue
                    
                    combined_input, true_label = parsed_data
                    
                    # Run fake news detection
                    detection_result = detect_fake_news(client, combined_input)
                    
                    # Extract prediction
                    predicted_label = extract_assessment(detection_result["verification"])
                    
                    # Calculate if prediction is correct
                    correct = (predicted_label == true_label) if predicted_label is not None else False
                    if correct:
                        correct_predictions += 1
                    
                    # Store result
                    result = {
                        'text': combined_input,
                        'true_label': true_label,
                        'predicted_label': predicted_label,
                        'correct': correct,
                        'facts': detection_result["facts_extracted"],
                        'verification': detection_result["verification"]
                    }
                    batch_results.append(result)
                    total_processed += 1
                
                # Write batch results to CSV
                for result in batch_results:
                    writer.writerow(result)
                    results.append(result)
                
                # Sleep to avoid rate limiting
                if i + batch_size < len(lines):
                    time.sleep(2)
    
    # Calculate accuracy
    accuracy = correct_predictions / total_processed if total_processed > 0 else 0
    
    print(f"\nProcessed {total_processed} samples")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.4f}")
    
    return {
        'results': results,
        'accuracy': accuracy,
        'correct_predictions': correct_predictions,
        'total_processed': total_processed
    }

In [11]:
dataset_path = r'C:\Users\CoolA\Code\dataset_2way_output.txt'
output_path="results/fakkedit_results.csv"
num_samples=500

print(f"Starting processing of {num_samples} samples from {dataset_path}")
start_time = time.time()

metrics = process_dataset(client, dataset_path, output_path, num_samples)

end_time = time.time()
processing_time = end_time - start_time

print(f"\nProcessing completed in {processing_time:.2f} seconds")
print(f"Results saved to {output_path}")

Starting processing of 500 samples from C:\Users\CoolA\Code\dataset_2way_output.txt


Processing dataset:  38%|███▊      | 19/50 [21:45<35:30, 68.72s/it]


UnicodeEncodeError: 'charmap' codec can't encode character '\u011f' in position 147: character maps to <undefined>

## Ultra-fast Agentic FND

In [13]:
import pandas as pd
import io
import time
from tqdm import tqdm
import csv
import os
import concurrent.futures
import re
from functools import lru_cache
import threading

def agent1_factbase(client, news_text):
    """
    Agent 1: Extract relevant facts from the input news using Gemini 1.5 Pro
    Returns facts in CSV format to be processed by Agent 2
    """
    start_time = time.time()
    
    # Create a simpler prompt for faster processing
    prompt = f"""
    Extract verifiable claims from this text as CSV with headers "claim,source,confidence":
    {news_text}
    """
    
    try:
        response = client.models.generate_content(
            model='gemini-1.5-pro',
            contents=prompt
        )
        result = response.text
        
        # Ensure it's valid CSV format
        if not "claim,source,confidence" in result:
            result = "claim,source,confidence\n" + result.replace('\n', ' ').strip()
            
    except Exception as e:
        print(f"Error in Agent 1: {e}")
        result = "claim,source,confidence\nError processing input,agent1,0.0"
        
    end_time = time.time()
    processing_time = end_time - start_time
    
    # Print timing only if it's exceptionally slow
    if processing_time > 10:
        print(f"⚠️ Agent 1 took {processing_time:.2f}s")
        
    return result

# Use a regex pattern to extract assessment value
ASSESSMENT_PATTERN = re.compile(r'<overall_assessment>([01]|REAL|FAKE)</overall_assessment>')

def agent2_verifier(client, news_text, facts_csv):
    """
    Agent 2: Analyze the news and extracted facts to determine if the news is fake
    Uses Gemini 2.0 Flash for faster reasoning
    Returns analysis in XML format with binary classification (1=REAL, 0=FAKE)
    """
    start_time = time.time()
    
    # Create a more compact facts representation
    try:
        facts_df = pd.read_csv(io.StringIO(facts_csv))
        # Limit to top 3 facts for efficiency
        facts_df = facts_df.head(3)
        facts_formatted = "; ".join([
            f"Claim: {row['claim']} (Source: {row['source']}, Conf: {row['confidence']})"
            for _, row in facts_df.iterrows()
        ])
    except:
        facts_formatted = facts_csv
    
    # Create a simpler prompt for faster processing
    prompt = f"""
    Classify if this news is real (1) or fake (0) based on these facts. Return in XML:
    <verification>
      <overall_assessment>1 for REAL or 0 for FAKE</overall_assessment>
      <confidence_score>0-1</confidence_score>
      <key_issues>Brief issues</key_issues>
      <reasoning>Brief reasoning</reasoning>
    </verification>
    
    News: {news_text}
    Facts: {facts_formatted}
    """
    
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=prompt
        )
        result = response.text
        
        # Ensure it has the required XML format
        if not "<verification>" in result:
            result = f"""<verification>
  <overall_assessment>0</overall_assessment>
  <confidence_score>0.5</confidence_score>
  <key_issues>Malformed response</key_issues>
  <reasoning>Could not properly analyze the content</reasoning>
</verification>"""
            
    except Exception as e:
        print(f"Error in Agent 2: {e}")
        result = """<verification>
  <overall_assessment>0</overall_assessment>
  <confidence_score>0.0</confidence_score>
  <key_issues>Error processing input</key_issues>
  <reasoning>Agent 2 encountered an error</reasoning>
</verification>"""
        
    end_time = time.time()
    processing_time = end_time - start_time
    
    # Print timing only if it's exceptionally slow
    if processing_time > 10:
        print(f"⚠️ Agent 2 took {processing_time:.2f}s")
        
    return result

@lru_cache(maxsize=1024)
def detect_fake_news(client, news_text):
    """
    Main function to orchestrate the 2-agent system for fake news detection
    Added caching to avoid reprocessing identical entries
    """
    start_time = time.time()
    
    # Step 1: Use Agent 1 to extract facts
    facts_csv = agent1_factbase(client, news_text)
    
    # Step 2: Use Agent 2 to verify and reason about the news
    verification_result = agent2_verifier(client, news_text, facts_csv)
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Print total time if it's exceptionally slow
    if total_time > 30:
        print(f"⚠️ Total processing took {total_time:.2f}s for: {news_text[:50]}...")
        
    return {
        "facts_extracted": facts_csv,
        "verification": verification_result,
        "processing_time": total_time
    }

def parse_dataset_line(line):
    """
    Parse a line from the Uni-Fakkedit-55k dataset
    Returns a tuple of (text, objects, label) or None if parsing fails
    """
    try:
        # Extract text
        text_parts = line.split("[TEXT]")
        if len(text_parts) != 2:
            return None
        after_text = text_parts[1]

        # Extract objects
        text_objects_parts = after_text.split("[OBJECTS]")
        if len(text_objects_parts) != 2:
            return None
        text_part = text_objects_parts[0].strip()
        after_objects = text_objects_parts[1]

        # Extract label
        objects_label_parts = after_objects.split("[LABEL]")
        if len(objects_label_parts) != 2:
            return None
        objects_part = objects_label_parts[0].strip()
        label_part = objects_label_parts[1].strip()

        # Combine text and objects - keep it short
        combined_input = f"{text_part}. Objects: {objects_part}"
        label_int = int(label_part)
        
        return (combined_input, label_int)
    except Exception as e:
        print(f"Error parsing line: {e}")
        return None

def extract_assessment(verification_xml):
    """
    Extract the overall assessment (1 or 0) from the verification XML
    Returns the assessment value or None if extraction fails
    Uses regex for faster extraction
    """
    try:
        # Use regex pattern for faster extraction
        match = ASSESSMENT_PATTERN.search(verification_xml)
        if match:
            assessment_str = match.group(1)
            if assessment_str == "1" or assessment_str.upper() == "REAL":
                return 1
            elif assessment_str == "0" or assessment_str.upper() == "FAKE":
                return 0
        
        # Fallback to simple string check
        if "<overall_assessment>1</overall_assessment>" in verification_xml:
            return 1
        elif "<overall_assessment>0</overall_assessment>" in verification_xml:
            return 0
        
        return None
    except Exception as e:
        print(f"Error extracting assessment: {e}")
        return None

def sanitize_for_csv(text):
    """
    Sanitize text to ensure it can be written to CSV safely
    Handles Unicode characters and other potential encoding issues
    """
    if text is None:
        return ""
    
    # Replace problematic characters that might cause issues
    return str(text).replace('\r', ' ').replace('\n', ' ')

def process_batch(client, batch_lines, lock, output_path, writer=None):
    """
    Process a batch of dataset lines
    Returns a list of results for the batch
    """
    batch_results = []
    batch_timings = []
    
    for line in batch_lines:
        parsed_data = parse_dataset_line(line)
        if parsed_data is None:
            continue
        
        combined_input, true_label = parsed_data
        
        # Process start time
        entry_start_time = time.time()
        
        # Run fake news detection
        detection_result = detect_fake_news(client, combined_input)
        
        # Extract prediction
        predicted_label = extract_assessment(detection_result["verification"])
        
        # Calculate if prediction is correct
        correct = (predicted_label == true_label) if predicted_label is not None else False
        
        # Record processing time
        entry_time = time.time() - entry_start_time
        batch_timings.append(entry_time)
        
        # Sanitize data for CSV output
        safe_text = sanitize_for_csv(combined_input)
        safe_facts = sanitize_for_csv(detection_result["facts_extracted"])
        safe_verification = sanitize_for_csv(detection_result["verification"])
        
        # Store result
        result = {
            'text': safe_text,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'correct': correct,
            'facts': safe_facts,
            'verification': safe_verification,
            'processing_time': entry_time
        }
        batch_results.append(result)
        
        # Write result to CSV if provided with a writer
        if writer:
            with lock:
                try:
                    writer.writerow(result)
                except Exception as e:
                    print(f"Error writing to CSV: {e}")
                    # Try writing a simplified version
                    simple_result = {
                        'text': 'Unicode error - text removed',
                        'true_label': true_label,
                        'predicted_label': predicted_label,
                        'correct': correct,
                        'facts': 'Unicode error - text removed',
                        'verification': 'Unicode error - text removed',
                        'processing_time': entry_time
                    }
                    writer.writerow(simple_result)
    
    return batch_results, batch_timings

def process_dataset(client, dataset_path, output_path, num_samples=500, batch_size=5, max_workers=2):
    """
    Process the first num_samples entries from the Uni-Fakkedit-55k dataset
    Saves results to a CSV file and returns evaluation metrics
    Uses parallel processing for faster execution
    """
    results = []
    processing_times = []
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Read the dataset lines
    with open(dataset_path, 'r', encoding='utf-8') as f:
        lines = []
        for _ in range(num_samples):
            line = f.readline()
            if not line:
                break  # End of file
            lines.append(line)
    
    # Create CSV file and writer with UTF-8 encoding
    csvfile = open(output_path, 'w', newline='', encoding='utf-8')
    fieldnames = ['text', 'true_label', 'predicted_label', 'correct', 'facts', 'verification', 'processing_time']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # Create a thread lock for CSV writing
    lock = threading.Lock()
    
    # Process batches with ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit batch processing tasks
        futures = []
        for i in range(0, len(lines), batch_size):
            batch_lines = lines[i:i+batch_size]
            futures.append(executor.submit(process_batch, client, batch_lines, lock, output_path, writer))
        
        # Process results as they complete
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
            batch_results, batch_timings = future.result()
            results.extend(batch_results)
            processing_times.extend(batch_timings)
    
    # Close CSV file
    csvfile.close()
    
    # Calculate metrics
    correct_predictions = sum(1 for result in results if result['correct'])
    total_processed = len(results)
    accuracy = correct_predictions / total_processed if total_processed > 0 else 0
    avg_processing_time = sum(processing_times) / len(processing_times) if processing_times else 0
    max_processing_time = max(processing_times) if processing_times else 0
    min_processing_time = min(processing_times) if processing_times else 0
    
    # Print results
    print(f"\nProcessed {total_processed} samples")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average processing time per entry: {avg_processing_time:.2f}s")
    print(f"Min processing time: {min_processing_time:.2f}s")
    print(f"Max processing time: {max_processing_time:.2f}s")
    
    return {
        'results': results,
        'accuracy': accuracy,
        'correct_predictions': correct_predictions,
        'total_processed': total_processed,
        'avg_processing_time': avg_processing_time,
        'min_processing_time': min_processing_time,
        'max_processing_time': max_processing_time
    }

dataset_path = r"C:\Users\CoolA\Code\dataset_2way_output.txt"

print(f"Starting processing of {num_samples} samples from {dataset_path}")
print(f"Output will be saved to {output_path}")

# Make sure the output directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Run a quick test to measure base performance
print("Running performance test...")
test_news = "Scientists discovered a new type of battery that can be charged in seconds."
start_time = time.time()
test_result = detect_fake_news(client, test_news)
test_time = time.time() - start_time
print(f"Test processing time: {test_time:.2f}s")

# Start main processing
start_time = time.time()
metrics = process_dataset(client, dataset_path, output_path, num_samples)
end_time = time.time()

total_time = end_time - start_time
entries_per_second = metrics['total_processed'] / total_time if total_time > 0 else 0

print(f"\nProcessing completed in {total_time:.2f} seconds")
print(f"Average speed: {entries_per_second:.2f} entries per second")
print(f"Results saved to {output_path}")

Starting processing of 500 samples from C:\Users\CoolA\Code\dataset_2way_output.txt
Output will be saved to results/fakkedit_results.csv
Running performance test...
Test processing time: 8.64s


Processing batches:  11%|█         | 11/100 [03:11<30:42, 20.70s/it]

⚠️ Agent 1 took 11.05s


Processing batches: 100%|██████████| 100/100 [27:34<00:00, 16.54s/it]


Processed 500 samples
Correct predictions: 334
Accuracy: 0.6680
Average processing time per entry: 6.57s
Min processing time: 3.13s
Max processing time: 14.64s

Processing completed in 1654.12 seconds
Average speed: 0.30 entries per second
Results saved to results/fakkedit_results.csv


# Findings

1. 5-piece FND: **72.86%** accuracy, 6.55 eps
2. 3-piece FND: **66.80%** accuracy, 0.30 eps